<a href="https://colab.research.google.com/github/disha2sinha/Movie-Recommendation-System/blob/master/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from ast import literal_eval
import pandas as pd
import math
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import os
from sklearn.cluster import KMeans
import math

## **IMPORTING MOVIES DATASET WHICH WAS PREVIOUSLY ARRANGED ACCORDING TO POPULARITY**

In [2]:
movies_data = pd.read_csv('https://raw.githubusercontent.com/disha2sinha/Movie-Recommendation-System/master/Revised%20Datasets/PopularMovies.csv?token=AKDEMFKNH37ZL3MQ4B2ZATTABWFE4',engine='python',index_col=0)
movies_data.head()

,tmdbId,title,budget,profit,score,release_year,release_day,keywords,genres,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,211672,Minions,74000000,1.082731e+09,0.625217,2015,Wednesday,"['assistant', 'aftercreditsstinger', 'duringcr...",Family|Animation|Adventure|Comedy,United States of America,Universal Pictures|Illumination Entertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda
1,550,Fight Club,63000000,3.785375e+07,0.554807,1999,Friday,"['support group', 'dual identity', 'nihilism',...",Drama,Germany|United States of America,Twentieth Century Fox Film Corporation|Regency...,English,139.0,A ticking-time-bomb insomniac and a slippery s...,Released,Edward Norton|Brad Pitt|Meat Loaf|Jared Leto|H...,David Fincher
2,155,The Dark Knight,185000000,8.195584e+08,0.541935,2008,Wednesday,"['dc comics', 'crime fighter', 'secret identit...",Drama|Action|Crime|Thriller,United Kingdom|United States of America,DC Comics|Legendary Pictures|Warner Bros.|DC E...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan
3,680,Pulp Fiction,8000000,2.059288e+08,0.541717,1994,Saturday,"['transporter', 'brothel', 'drug dealer', 'box...",Thriller|Crime,United States of America,Miramax Films|A Band Apart|Jersey Films,English,154.0,"A burger-loving hit man, his philosophical par...",Released,John Travolta|Samuel L. Jackson|Uma Thurman|Br...,Quentin Tarantino
4,244786,Whiplash,3300000,9.792000e+06,0.528110,2014,Friday,"['jazz', 'obsession', 'conservatory', 'music t...",Drama,United States of America,Bold Films|Blumhouse Productions|Right of Way ...,English,105.0,"Under the direction of a ruthless instructor, ...",Released,Miles Teller|J.K. Simmons|Melissa Benoist|Aust...,Damien Chazelle


In [3]:
movies_data.shape

(9081, 17)

In [4]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9081 entries, 0 to 9080
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                9081 non-null   int64  
 1   title                 9081 non-null   object 
 2   budget                9081 non-null   int64  
 3   profit                9081 non-null   float64
 4   score                 9081 non-null   float64
 5   release_year          9081 non-null   int64  
 6   release_day           9081 non-null   object 
 7   keywords              9081 non-null   object 
 8   genres                9046 non-null   object 
 9   production_countries  8787 non-null   object 
 10  production_companies  8417 non-null   object 
 11  original_language     9079 non-null   object 
 12  runtime               9081 non-null   float64
 13  content               9069 non-null   object 
 14  status                9081 non-null   object 
 15  cast                 

In [5]:
movies_data=movies_data.replace(np.nan,'Unknown',regex=True)
movies_data['release_year']=movies_data['release_year'].replace('NaT','1874',regex=True)

In [6]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9081 entries, 0 to 9080
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                9081 non-null   int64  
 1   title                 9081 non-null   object 
 2   budget                9081 non-null   int64  
 3   profit                9081 non-null   float64
 4   score                 9081 non-null   float64
 5   release_year          9081 non-null   int64  
 6   release_day           9081 non-null   object 
 7   keywords              9081 non-null   object 
 8   genres                9081 non-null   object 
 9   production_countries  9081 non-null   object 
 10  production_companies  9081 non-null   object 
 11  original_language     9081 non-null   object 
 12  runtime               9081 non-null   float64
 13  content               9081 non-null   object 
 14  status                9081 non-null   object 
 15  cast                 

#**CONTENT-BASED RECOMMENDATION SYSTEM**

## **KEYWORDS ARE FORMED FROM ALL THE NON-NUMERIC COLUMNS**

In [7]:
movies_data['production_companies']=movies_data['production_companies'].str.replace(" ","")
movies_data['production_companies']=movies_data['production_companies'].str.replace("|","  ")
movies_data['production_countries']=movies_data['production_countries'].str.replace(" ","")
movies_data['production_countries']=movies_data['production_countries'].str.replace("|","  ")
movies_data['cast']=movies_data['cast'].str.replace(" ","")
movies_data['cast']=movies_data['cast'].str.replace("|","  ")
movies_data['genres']=movies_data['genres'].str.replace("|"," ")
movies_data['director']=movies_data['director'].str.replace(" ","")
movies_data['keywords']=movies_data['keywords'].apply(lambda x:x[1:-1])
movies_data['keywords']=movies_data['keywords'].str.replace(',',"  ")
movies_data['keywords']=movies_data['keywords'].str.replace("'","")
movies_data.head(10)

,tmdbId,title,budget,profit,score,release_year,release_day,keywords,genres,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,211672,Minions,74000000,1.082731e+09,0.625217,2015,Wednesday,assistant aftercreditsstinger duringcredit...,Family Animation Adventure Comedy,UnitedStatesofAmerica,UniversalPictures IlluminationEntertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,SandraBullock JonHamm MichaelKeaton Allison...,KyleBalda
1,550,Fight Club,63000000,3.785375e+07,0.554807,1999,Friday,support group dual identity nihilism rag...,Drama,Germany UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation RegencyEnt...,English,139.0,A ticking-time-bomb insomniac and a slippery s...,Released,EdwardNorton BradPitt MeatLoaf JaredLeto H...,DavidFincher
2,155,The Dark Knight,185000000,8.195584e+08,0.541935,2008,Wednesday,dc comics crime fighter secret identity ...,Drama Action Crime Thriller,UnitedKingdom UnitedStatesofAmerica,DCComics LegendaryPictures WarnerBros. DCEn...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,ChristianBale MichaelCaine HeathLedger Aaro...,ChristopherNolan
3,680,Pulp Fiction,8000000,2.059288e+08,0.541717,1994,Saturday,transporter brothel drug dealer boxer ...,Thriller Crime,UnitedStatesofAmerica,MiramaxFilms ABandApart JerseyFilms,English,154.0,"A burger-loving hit man, his philosophical par...",Released,JohnTravolta SamuelL.Jackson UmaThurman Bru...,QuentinTarantino
4,244786,Whiplash,3300000,9.792000e+06,0.528110,2014,Friday,jazz obsession conservatory music teache...,Drama,UnitedStatesofAmerica,BoldFilms BlumhouseProductions RightofWayFilms,English,105.0,"Under the direction of a ruthless instructor, ...",Released,MilesTeller J.K.Simmons MelissaBenoist Aust...,DamienChazelle
5,177572,Big Hero 6,165000000,4.871054e+08,0.523318,2014,Friday,brother brother relationship hero talent ...,Adventure Family Animation Action Comedy,UnitedStatesofAmerica,WaltDisneyPictures WaltDisneyAnimationStudios,English,102.0,The special bond that develops between plus-si...,Released,ScottAdsit RyanPotter DanielHenney T.J.Mill...,ChrisWilliams
6,210577,Gone Girl,61000000,3.083304e+08,0.515822,2014,Wednesday,based on novel marriage crisis disappearan...,Mystery Thriller Drama,UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation RegencyEnt...,English,145.0,With his wife's disappearance having become th...,Released,BenAffleck RosamundPike CarrieCoon NeilPatr...,DavidFincher
7,13,Forrest Gump,55000000,6.229454e+08,0.515705,1994,Wednesday,vietnam veteran hippie mentally disabled ...,Comedy Drama Romance,UnitedStatesofAmerica,ParamountPictures,English,142.0,A man with a low IQ has accomplished great thi...,Released,TomHanks RobinWright GarySinise MykeltiWill...,RobertZemeckis
8,11,Star Wars,11000000,7.643980e+08,0.495943,1977,Wednesday,android galaxy hermit death star light...,Adventure Action Science Fiction,UnitedStatesofAmerica,Lucasfilm TwentiethCenturyFoxFilmCorporation,English,121.0,Princess Leia is captured and held hostage by ...,Released,MarkHamill HarrisonFord CarrieFisher PeterC...,GeorgeLucas
9,278,The Shawshank Redemption,25000000,3.341469e+06,0.492673,1994,Friday,prison corruption police brutality priso...,Drama Crime,UnitedStatesofAmerica,CastleRockEntertainment WarnerBros.,English,142.0,Framed in the 1940s for the double murder of h...,Released,TimRobbins MorganFreeman BobGunton ClancyBr...,FrankDarabont


In [8]:
movies_data['keywords']=movies_data['keywords']+" " +movies_data['cast']+" "+movies_data['production_companies']+" "+movies_data['production_countries']+" "+movies_data['genres']+" "+movies_data['original_language']+" "+movies_data['release_day']+" "+movies_data['content']+" "+movies_data['director']
movies_data['keywords']=movies_data['keywords'].replace({"[^A-Za-z0-9]+": " "},regex=True)
movies_data['keywords'][0]

'assistant aftercreditsstinger duringcreditsstinger evil mastermind minions 3d SandraBullock JonHamm MichaelKeaton AllisonJanney SteveCoogan JenniferSaunders GeoffreyRush SteveCarell PierreCoffin KatyMixon MichaelBeattie HiroyukiSanada DaveRosenbaum AlexDowding PaulThornley KyleBalda AvaAcres UniversalPictures IlluminationEntertainment UnitedStatesofAmerica Family Animation Adventure Comedy English Wednesday Minions Stuart Kevin and Bob are recruited by Scarlet Overkill a super villain who alongside her inventor husband Herb hatches a plot to take over the world Before Gru they had a history of bad bosses KyleBalda'

## **WORD VECTORIZER IS CREATED FROM THE KEYWORDS AND A NEW DATAFRAME IS FORMED COMBINING THE NUMERIC COLUMNS AND WORD VECTORIZER FOR EACH MOVIE ID**

In [9]:
vectorizer=TfidfVectorizer(analyzer='word',stop_words='english')
X=vectorizer.fit_transform(movies_data['keywords'])
tfidf_df=pd.DataFrame.sparse.from_spmatrix(X)

In [10]:
movies_df=movies_data[['tmdbId','budget','profit','score','release_year','runtime']]
movies_df=movies_df.join(tfidf_df)
movies_df.set_index('tmdbId',inplace=True)
movies_df.shape

(9081, 124699)

In [11]:
dftomat=scipy.sparse.csr_matrix(movies_df.values)

In [12]:
dftomat[1,:].toarray()

array([[6.30000000e+07, 3.78537530e+07, 5.54807458e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [13]:
dftomat.shape

(9081, 124699)

## **PERFORMING KMeans ON THE NEW DATAFRAME TO FORM CLUSTERS OF SIMILAR MOVIES:**

### **FINDING OPTIMAL K USING ELBOW METHOD**

In [14]:
#sse={}
#for k in range(4,15):
  #print("K=",k)
  #kmeans=KMeans(n_clusters=k,init='random',max_iter=100).fit(dftomat)
  #sse[k]=kmeans.inertia_
#plt.plot(list(sse.keys()),list(sse.values()),'bx-')
#plt.xlabel("Number of Cluster")
#plt.ylabel("SSE")
#plt.savefig("elbow_method.png")

### **OPTIMAL K VALUE IS 6**

In [15]:
kmeans=KMeans(n_clusters=6,init='k-means++',max_iter=500).fit(dftomat)
movies_df['KmeansCluster']=kmeans.labels_

In [16]:
print("Cluster 0: ",movies_df[movies_df['KmeansCluster']==0].KmeansCluster.count())
print("Cluster 1: ",movies_df[movies_df['KmeansCluster']==1].KmeansCluster.count())
print("Cluster 2: ",movies_df[movies_df['KmeansCluster']==2].KmeansCluster.count())
print("Cluster 3: ",movies_df[movies_df['KmeansCluster']==3].KmeansCluster.count())
print("Cluster 4: ",movies_df[movies_df['KmeansCluster']==4].KmeansCluster.count())
print("Cluster 5: ",movies_df[movies_df['KmeansCluster']==5].KmeansCluster.count())

Cluster 0:  7742
Cluster 1:  56
Cluster 2:  309
Cluster 3:  7
Cluster 4:  118
Cluster 5:  849


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim=cosine_similarity(dftomat)

In [18]:
cosine_sim=pd.DataFrame(cosine_sim,index=movies_df.index,columns=movies_df.index)
cosine_sim.head(20)

tmdbId,211672,550,155,680,244786,177572,210577,13,11,278,27205,424,129,24428,1891,293660,150540,238,12445,118340,68718,16869,807,120467,857,19995,240,348,157336,121,122,77338,539,119450,269149,62,11324,280,49026,207703,...,18823,243688,1639,2698,44833,10200,209112,5491,96936,60304,7278,68728,8487,329833,314,76757,52449,9738,9760,81005,49849,2486,47933,8645,14161,205321,1250,87818,82700,71880,9480,10196,71676,216015,14164,44912,47964,241251,415,166424
tmdbId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
211672,1.000000,0.572282,0.988201,0.999568,0.967204,0.966809,0.991936,0.999804,0.998550,0.199758,0.985973,0.999987,0.999944,0.995058,0.999433,0.999933,0.983352,0.999069,0.999418,0.978773,0.973680,0.978923,0.999057,0.990722,0.994992,0.999702,0.957754,0.998832,0.970236,0.999694,0.999730,0.999323,0.999102,0.972185,0.994826,0.993928,0.958735,0.986336,0.975307,0.985563,...,0.702917,0.897524,0.096258,0.056781,0.471504,0.915805,0.951344,-0.395174,0.331497,0.977956,0.907367,0.861535,0.357554,0.185968,-0.108647,0.113349,0.999444,0.942431,0.975371,0.081929,0.140179,0.866843,0.844490,0.897146,0.963929,0.999299,0.778217,0.988024,0.708574,0.716007,0.831770,0.790525,0.884495,0.999976,-0.495926,0.166394,0.942730,0.999900,0.720256,0.433691
550,0.572282,1.000000,0.691134,0.547920,0.761810,0.762812,0.671599,0.588424,0.527314,0.917847,0.701128,0.576471,0.563562,0.650882,0.544356,0.581738,0.711767,0.536377,0.599919,0.728203,0.744127,0.727699,0.607339,0.678422,0.651383,0.592120,0.783945,0.611236,0.753835,0.592385,0.591169,0.541716,0.537028,0.748434,0.652635,0.659038,0.781810,0.699566,0.739262,0.702863,...,0.985550,0.875254,0.871336,0.851228,0.993012,0.853453,0.797123,0.527158,0.963398,0.730902,0.863972,0.909383,0.970467,0.912178,0.753026,0.879639,0.599306,0.813563,0.739067,0.864187,0.892182,0.904943,0.922485,0.875668,0.769905,0.541185,0.960352,0.691963,0.984166,0.982234,0.931237,0.954631,0.888778,0.577965,0.428299,0.903849,0.813043,0.583848,0.981070,0.987115
155,0.988201,0.691134,1.000000,0.983269,0.994696,0.994535,0.999644,0.991042,0.978524,0.347479,0.999903,0.988971,0.986522,0.998525,0.982485,0.989908,0.999581,0.980674,0.992850,0.998615,0.997100,0.998653,0.993918,0.999848,0.998561,0.991643,0.990502,0.994447,0.995879,0.991686,0.991491,0.981894,0.980825,0.996587,0.998648,0.999053,0.990968,0.999931,0.997626,0.999866,...,0.803565,0.954474,0.247577,0.209029,0.601012,0.966514,0.987313,-0.249813,0.472091,0.998399,0.961042,0.929131,0.496375,0.334268,0.044894,0.264190,0.992758,0.982530,0.997646,0.233613,0.290178,0.932980,0.916557,0.954219,0.993322,0.981775,0.865222,0.999999,0.808292,0.814483,0.906981,0.875000,0.945518,0.989241,-0.357070,0.315461,0.982696,0.990272,0.818010,0.566585
680,0.999568,0.547920,0.983269,1.000000,0.959316,0.958877,0.987781,0.998788,0.999701,0.170858,0.980638,0.999404,0.999823,0.991708,0.999991,0.999160,0.977584,0.999906,0.997983,0.972323,0.966556,0.972494,0.997349,0.986297,0.991623,0.998553,0.948883,0.996979,0.962695,0.998535,0.998615,0.999973,0.999916,0.964877,0.991408,0.990263,0.949960,0.981064,0.968391,0.980158,...,0.681697,0.884168,0.066947,0.027397,0.445368,0.903598,0.941871,-0.422016,0.303610,0.971393,0.894614,0.846232,0.329937,0.156994,-0.137832,0.084083,0.998031,0.932190,0.968463,0.052586,0.111002,0.851807,0.828376,0.883768,0.955685,0.999968,0.759413,0.983059,0.687517,0.695169,0.815086,0.772174,0.870393,0.999339,-0.521247,0.137325,0.932514,0.999050,0.699545,0.407006
244786,0.967204,0.761810,0.994696,0.959316,1.000000,0.999999,0.991596,0.972048,0.952130,0.442090,0.996031,0.968491,0.964458,0.987645,0.958106,0.970080,0.997256,0.955347,0.975304,0.998730,0.999639,0.998693,0.977317,0.992750,0.987748,0.973113,0.999391,0.978347,0.999925,0.973189,0.972841,0.957201,0.955575,0.999792,0.988004,0.989279,0.999505,0.995834,0.999418,0.996245,...,0.860528,0.980095,0.345924,0.308511,0.680036,0.987783,0.998409,-0.148886,0.560266,0.998921,0.984376,0.962236,0.583038,0.429441,0.147415,0.361997,0.975135,0.996461,

In [19]:
def content_based(movie_name):
  movie_idx=movies_data[movies_data['title']==movie_name]['tmdbId'].values[0]
  movie_cluster=movies_df.at[movie_idx,'KmeansCluster']
  bestRecommendationsIdx=movies_df[movies_df['KmeansCluster']==movie_cluster].index.values
  scores=[]
  for index in bestRecommendationsIdx:
    scores.append((index,cosine_sim.at[movie_idx,index]))
  scores.sort(key = lambda x: x[1],reverse=True)
  bestRecommendationsdf=pd.DataFrame(columns=['title','genres'])
  for tup in scores:
    if tup[0]!=movie_idx:
      df=movies_data[movies_data['tmdbId']==tup[0]][['title','genres']]
      bestRecommendationsdf=pd.concat([bestRecommendationsdf,df])

  return bestRecommendationsdf.head(10)
bestRecommendationsdf=content_based('Minions')
bestRecommendationsdf

,title,genres
180,Jurassic Park,Adventure Science Fiction
9009,The Twilight Saga: New Moon,Adventure Fantasy Drama Romance
72,The Lion King,Family Animation Drama
15,Deadpool,Action Adventure Comedy
134,Despicable Me 2,Animation Comedy Family
30,The Lord of the Rings: The Return of the King,Adventure Fantasy Action
29,The Lord of the Rings: The Two Towers,Adventure Fantasy Action
8987,The Secret Life of Pets,Animation Family
1010,Independence Day,Action Adventure Science Fiction
94,Finding Nemo,Animation Family


#**USER TO USER BASED COLLABORATIVE FILTERING**

In [20]:
ratings_data=pd.read_csv('https://raw.githubusercontent.com/disha2sinha/Movie-Recommendation-System/master/Revised%20Datasets/RevisedRatings.csv?token=AKDEMFPBZWMGMKXJLF375W3ACQQ26', index_col=0, engine='python')
ratings_data.head()

,rating,tmdbId
userId,,
1,2.5,9909
7,3.0,9909
31,4.0,9909
32,4.0,9909
36,3.0,9909


In [21]:
ratings_data['userId']=ratings_data.index
ratings_data=ratings_data.reset_index(drop=True)
movie_name=movies_data[['tmdbId','title']]
ratings_data=ratings_data.merge(movie_name,on='tmdbId')
ratings_data.head()

,rating,tmdbId,userId,title
0,2.5,9909,1,Dangerous Minds
1,3.0,9909,7,Dangerous Minds
2,4.0,9909,31,Dangerous Minds
3,4.0,9909,32,Dangerous Minds
4,3.0,9909,36,Dangerous Minds


In [22]:
print(" Unique Users :",ratings_data.userId.unique().shape[0])

 Unique Users : 671


In [26]:
def collaborative(user_profile):
  userId=user_profile.at[0,'userId']
  input_user=user_profile.drop(columns=['userId'])
  users = ratings_data[ratings_data['tmdbId'].isin(input_user['tmdbId'].tolist())]
  users=users[users['userId']!=userId]
  #Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
  userSubsetGroup = users.groupby(['userId'])

  #Sorting it so users with movie most in common with the input will have priority
  userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
  userSubsetGroup = userSubsetGroup[0:100]
  #Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
  pearsonCorDict = {}
  #For every user group in our subset
  for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='tmdbId')
    input_user = input_user.sort_values(by='tmdbId')
    #Get the N for the formula
    n = len(group)
    #Get the review scores for the movies that they both have in common
    temp = input_user[input_user['tmdbId'].isin(group['tmdbId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp['rating'].tolist()
    #put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorDict[name] = Sxy/math.sqrt(Sxx*Syy)
    else:
        pearsonCorDict[name] = 0
  pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
  pearsonDF.columns = ['similarityIndex']
  pearsonDF['userId'] = pearsonDF.index
  pearsonDF.index = range(len(pearsonDF))
  topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
  topUsersRating=topUsers.merge(ratings_data, left_on='userId', right_on='userId', how='inner')
  #Multiplies the similarity by the user's ratings
  topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
  #Applies a sum to the topUsers after grouping it up by userId
  tempTopUsersRating = topUsersRating.groupby('tmdbId').sum()[['similarityIndex','weightedRating']]
  tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
    #Creates an empty dataframe
  recommendation_df = pd.DataFrame()
    #Now we take the weighted average
  recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
  recommendation_df['tmdbId'] = tempTopUsersRating.index
  recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
  recommendations=pd.DataFrame(movies_data.loc[movies_data['tmdbId'].isin(recommendation_df.head(25)['tmdbId'].tolist())][['title','genres']])
  #Also Recommend Movies similar to liked movies of user
  liked_movies=user_profile[user_profile['rating']>=3.0]
  for movie in liked_movies['title']:
    rating=liked_movies[liked_movies['title']==movie]['rating'].values[0]
    print()
    print("====================Movie Similar to ",movie," rated ",rating," by User=======================================")
    print()
    print(content_based(movie))
  print()
  print("=============================User May Also Like=============================================")
  print()
  print(pd.DataFrame(recommendations))
collaborative(ratings_data[ratings_data['userId']==1])


====================Movie Similar to  Dumbo  rated  3.0  by User=======================================

                      title                                     genres
5683             Son in Law                       Comedy Drama Romance
8409              Guess Who                             Comedy Romance
8491        Jennifer's Body                              Comedy Horror
4145               Soapdish                       Comedy Drama Romance
7696           The Big Bang                    Action Mystery Thriller
8749  40 Days and 40 Nights                             Comedy Romance
8375                   They                            Horror Thriller
7979              Replicant  Action Adventure Science Fiction Thriller
7842               The Pest                              Action Comedy
6682                Carpool                              Comedy Family

====================Movie Similar to  Sleepers  rated  3.0  by User=======================================

    

## **NEW USER**

In [27]:
new_user=[[3.0,362045,800,'Bajirao Mastani'],[4.5,20453,800,'3 Idiots'],[3.0,9909,800,'Dangerous Minds'],[2.5,211672,800,'Minions'],[4.0,185008,800,'Yeh Jawaani Hai Deewani'],[5.0, 11, 800, 'Star Wars'],[3.0, 193, 800, 'Star Trek: Generations'],
       [4.0, 36955, 800, 'True Lies'],[2.0, 172, 800, 'Star Trek V: The Final Frontier'],[1.0, 9361, 800, 'The Last of the Mohicans']]
user_df=pd.DataFrame(new_user,columns=['rating','tmdbId','userId','title'])
user_df

,rating,tmdbId,userId,title
0,3.0,362045,800,Bajirao Mastani
1,4.5,20453,800,3 Idiots
2,3.0,9909,800,Dangerous Minds
3,2.5,211672,800,Minions
4,4.0,185008,800,Yeh Jawaani Hai Deewani
5,5.0,11,800,Star Wars
6,3.0,193,800,Star Trek: Generations
7,4.0,36955,800,True Lies
8,2.0,172,800,Star Trek V: The Final Frontier
9,1.0,9361,800,The Last of the Mohicans


In [28]:
collaborative(user_df)


====================Movie Similar to  Bajirao Mastani  rated  3.0  by User=======================================

                                    title                         genres
781                         Boogie Nights                          Drama
904                         King of Kings                          Drama
440                         Swimming Pool                 Thriller Crime
3391          One Piece Film Strong World    Animation Fantasy Adventure
3376         History of the World: Part I                         Comedy
5585                             Drumline     Drama Romance Comedy Music
8702           The Men Who Stare at Goats                     Comedy War
6990                   Eyes of Laura Mars  Drama Horror Mystery Thriller
416   Lock, Stock and Two Smoking Barrels                   Comedy Crime
320                               Amadeus            Drama History Music

====================Movie Similar to  3 Idiots  rated  4.5  by User=============